IMPORTING COMPRESSION AND OTHER RELATED PACKAGES

In [ ]:


from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
#from keras.applications.resnet50 import ResNet50
#from keras.applications.resnet50 import preprocess_input
from keras.models import Model
from os import listdir
import tqdm
import pickle as pk
import pathlib
import string 


import tensorflow as tf 
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
pip install -q tensorflow-model-optimization



from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils


import scipy as sp
import numpy as np
import pandas as pd
from numpy.random import rand

pd.options.display.max_colwidth = 600


from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split
from PIL import Image


import matplotlib.pyplot as plt

In [ ]:
folder_path = '###'


FEATURE EXTRACTION USING UNQUANTIZED MODEL

In [1]:

def extract_features(directory):
	# load the model
	model,features = VGG16(),dict()
 
	model = Model(inputs=model.input, outputs=model.layers[-4].output)  # Before all dense layers
 
	# extract features from each photo

	for name in tqdm(listdir(directory)):
		# load an image from file
		filename = directory + '/' + name
		
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)	
		# get image id
		image_id = name.split('.')[0]
	  # Store feature with its key in dictionary
		feature = feature.reshape(1,-1)
		features[image_id] = feature
	return features
 
# extract features from all images
directory = folder_path + '/Flicker8k_Dataset'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
pk.dump(features, open(folder_path + '#', 'wb'))

TRAINING AND SUBSEQUENT PRUNING OF CNN USING CIFAR100

In [ ]:
BATCH_SIZE = 32
EPOCHS = 50
NUM_CLASSES = 100
LEARNING_RATE = 1e-4
MOMENTUM = 0.9

In [11]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size=0.15, 
                                                  stratify=np.array(y_train), 
                                                  random_state=42)

In [ ]:
Y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
Y_val = np_utils.to_categorical(y_val, NUM_CLASSES)
Y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
X_train = np.array([np.array(Image.fromarray(x).resize((48, 48))) for x in X_train])
X_val = np.array([np.array(Image.fromarray(x).resize((48, 48))) for x in X_val])
X_test = np.array([np.array(Image.fromarray(x)).resize((48, 48)) for x in X_test])



In [2]:
base_model = ResNet50(weights='imagenet', 
                       include_top=False, 
                       input_shape=(48, 48, 3))

In [ ]:

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.50, 0),
    'block_size': (1, 1),
    'block_pooling_type': 'AVG'
}

FOR PRUNING RESNET-50

In [3]:

last = base_model.output
x = GlobalAveragePooling2D()(last)
x = Dropout(0.6)(x)
x= BatchNormalization()(x)
last_dense = tfmot.sparsity.keras.prune_low_magnitude(Dense(1000, activation='relu'),**pruning_params)(x)

pred = Dense(NUM_CLASSES, activation='softmax')(last_dense)
model = Model(base_model.input, pred)

FOR PRUNING VGG-16

In [ ]:
last = base_model.get_layer('block3_pool').output
x = GlobalAveragePooling2D()(last)
x= BatchNormalization()(x)
x = tfmot.sparsity.keras.prune_low_magnitude(Dense(256, activation='relu'),**pruning_params)(x)

x = tfmot.sparsity.keras.prune_low_magnitude(Dense(256, activation='relu'),**pruning_params)(x)
last_dense = Dropout(0.6)(x)
pred = Dense(NUM_CLASSES, activation='softmax')(last_dense)
model = Model(base_model.input, pred)

In [ ]:
for layer in base_model.layers:
     layer.trainable = False


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=LEARNING_RATE),
              metrics=['accuracy'])

In [4]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=False)

In [ ]:
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train,
                                     Y_train, 
                                     batch_size=BATCH_SIZE)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1. / 255,
    horizontal_flip=False)

val_datagen.fit(X_val)
val_generator = val_datagen.flow(X_val,
                                 Y_val,
                                 batch_size=BATCH_SIZE)

In [5]:
train_steps_per_epoch = X_train.shape[0] // BATCH_SIZE
val_steps_per_epoch = X_val.shape[0] // BATCH_SIZE
callback = [tfmot.sparsity.keras.UpdatePruningStep()]
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=val_generator,
                              validation_steps=val_steps_per_epoch,
                              epochs= EPOCHS,
                              verbose = 1,
                              callbacks = callback)
pruned_model = tfmot.sparsity.keras.strip_pruning(model)

pruned_model.save(folder_path + '#' + '.h5')


FEATURE EXTRACTION USING QUANTIZED MODEL

In [6]:
def getQuantizedModel(model):

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  
  tflite_model = converter.convert()
  
  tflite_models_dir = pathlib.Path("#")
  tflite_models_dir.mkdir(exist_ok=True, parents=True)

  tflite_model_file = tflite_models_dir/"no_Q_model.tflite"
  tflite_model_file.write_bytes(tflite_model)

  
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  tflite_quant_model = converter.convert()
  tflite_model_quant_file = tflite_models_dir/"Q_model.tflite"
  tflite_model_quant_file.write_bytes(tflite_quant_model)

  interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
  interpreter_quant.allocate_tensors()

 
  return interpreter_quant

In [9]:
# extract features from each photo in the directory
def extract_features(directory,pruned_model,unpruned_model):
	# load the model
	model,features,base_model = pruned_model,dict(), unpruned_model
  
	model, pruned_model = Model(inputs= base_model.inputs, outputs = last_dense), tfmot.sparsity.keras.strip_pruning(model)
  
	quantized_model = getQuantizedModel(model)
 
	input_index = quantized_model.get_input_details()[0]["index"]
	
	output_index = quantized_model.get_output_details()[0]["index"]
 
	# extract features from each photo

	for name in tqdm(listdir(directory)):
		# load an image from file
		filename = directory + '/' + name
		
		image = load_img(filename, target_size=(48, 48))
    
		# convert the image pixels to a numpy array
		image = img_to_array(image)

    
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	
		# prepare the image for the VGG model 
		image = preprocess_input(image)
		# get features

		quantized_model.set_tensor(input_index, image)
	
		# Run inference.
		feature = quantized_model.invoke()
	
		# get image id
		image_id = name.split('.')[0]
		feature = quantized_model.get_tensor(output_index)[0]
    
	  # Store feature with its key in dictionary
		feature = feature.reshape(1,-1)
	
		features[image_id] = feature
		

	return features
 
# extract features from all images
directory = folder_path + '/Flicker8k_Dataset'

print(pruned_model.summary())


In [10]:
features = extract_features(directory,model,base_model)
print('Extracted Features: %d' % len(features))
# save to file
pk.dump(features, open(folder_path + '#', 'wb'))